In [1]:
import os
from bs4 import BeautifulSoup

In [2]:
slozka = "filmy"

In [3]:
def seznam_souboru(slozka):
    seznam = []
    for file in os.listdir(slozka):
        seznam.append(file)
    return(seznam)

In [4]:
seznam_filmu = seznam_souboru(slozka)
seznam_filmu[0:10]

['10144-tabor-padlych-zen.html',
 '10173-eastern-krvavy-hugo.html',
 '102282-o-jecminkovi.html',
 '102314-zimni-vila.html',
 '10242-narkoblues.html',
 '103341-jan-hus-mse-za-tri-mrtve-muze.html',
 '103354-obcansky-prukaz.html',
 '103369-kuky-se-vraci.html',
 '103372-habermannuv-mlyn.html',
 '103374-dobre-placena-prochazka.html']

## Začínáme scrapovat

### Nejdřív zjistíme, co všechno budeme scrapovat

In [123]:
kolik = None

(Čím menší číslo, tím menší vzorek a tím rychlejší test. Pro zpracování kompletního seznamu nutno změnit proměnou na None.)

In [124]:
def jake_jsou_sekce(slozka):
    sekce = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-section')
            for item_detail in item_details:
                h2_tags = item_detail.find_all('h2')
                for h2_tag in h2_tags:
                    if h2_tag.get_text() not in sekce:
                        sekce.append(h2_tag.get_text())
    return(sekce)

In [125]:
sekce = jake_jsou_sekce("filmy")

In [126]:
def jake_jsou_detaily(slozka):
    detaily = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='item-details')
            for item_detail in item_details:
                h3_tags = item_detail.find_all('h3')
                for h3_tag in h3_tags:
                    if h3_tag.get_text() not in detaily:
                        detaily.append(h3_tag.get_text())
    return(detaily)

In [127]:
detaily = jake_jsou_detaily(slozka)

In [128]:
def jake_jsou_kredity(slozka):
    kredity = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Štáb a tvůrci" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in kredity:
                            kredity.append(h3_tag.get_text())
    return(kredity)

In [129]:
kredity = jake_jsou_kredity(slozka)

In [130]:
def jake_jsou_technicke_detaily(slozka):
    technika = []
    for s in seznam_souboru(slozka)[:kolik]:
        with open(os.path.join(slozka, s), "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")
            item_details = soup.find_all(class_='db-film-credits-section')
            for item_detail in item_details:
                if "Technické údaje" in item_detail.text:
                    h3_tags = item_detail.find_all('h3')
                    for h3_tag in h3_tags:
                        if h3_tag.get_text() not in technika:
                            technika.append(h3_tag.get_text())
    return(technika)

In [131]:
technika = jake_jsou_technicke_detaily(slozka)

In [132]:
technika

['Délka',
 'Minutáž',
 'Distribuční nosič',
 'Poměr stran',
 'Barva',
 'Zvuk',
 'Zvukový systém/formát',
 'Verze',
 'Mluveno',
 'Podtitulky',
 'Úvodní/závěrečné titulky',
 'Původní metráž',
 'Mezititulky',
 'Animační technika',
 'Natáčecí formát']

## Scrapování filmů

In [133]:
def id_persony(url):
    if "cs/person/" in url:
        id_persony = url.split("/")[-2]
    else:
        id_persony = 0
    return(id_persony)

In [134]:
def pridej_atributy(slovnik, seznam, nalezene):
    for s in seznam:
        for n in nalezene:
            if s in n:
                n = n.replace(s,"")
                slovnik[s] = n.strip()

In [136]:
def scrape_film(slozka, soubor):

    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    
    atributy = {}
    
    ## Detaily – země původu, copyright…
    
    item_details = soup.find_all(class_='item-details')
    divs_text = []

    for item_detail in item_details:
        divs = item_detail.find_all('div')
        for div in divs:
            if div.find_all("span"):
                prvni = div.find("h3").text
                odkazy = div.find_all("a")
                odkazy2 = []
                for a in odkazy:
                    id = id_persony(a["href"])
                    odkazy2.append(f"{a.text} ({id})")
                    druhy = ", ".join(odkazy2)
                    divs_text.append(prvni + " " + druhy)
            else:
                divs_text.append(div.text)    

    pridej_atributy(atributy, detaily, divs_text)
    
    ## Credits
    
    item_details = soup.find_all(class_='db-film-credits-section')
    divs_text = []

    for item_detail in item_details:
        
        if "Štáb a tvůrci" in item_detail.text:
            divs = item_detail.find_all('div')
            for div in divs:
                if div.find_all("a"):
                    prvni = div.find("h3").text
                    odkazy = div.find_all("a")
                    odkazy2 = []
                    for a in odkazy:
                        id = id_persony(a["href"])
                        odkazy2.append(f"{a.text} ({id})")
                        druhy = ", ".join(odkazy2)
                        divs_text.append(prvni + " " + druhy)
                else:
                    divs_text.append(div.text)
            
            pridej_atributy(atributy, kredity, divs_text)
            
        if "Technické údaje" in item_detail.text or "Produkční údaje" in item_detail.text:
            divs = item_detail.find_all('div')
            for div in divs:
                if div.find("h3"):
                    if div.find("p"):
                        atributy[div.find("h3").text.strip()] = div.find("p").text.strip()
                        
        if "Písně" in item_detail.text: ## Bacha, "písně" je dost generický text!
            pisne = item_detail.find_all('h3')
            pisne = [pisen.text.strip() for pisen in pisne]
            atributy["Písně"] = pisne

    item_details = soup.find_all(class_='film-description')
    
    for item_detail in item_details:
    
        if "Anotace" in item_detail.text:
            anotace = item_detail.find('p')            
            atributy["Anotace"] = anotace.text.strip()
            
        if "Obsah" in item_detail.text:
            obsah = item_detail.find('p')            
            atributy["Obsah"] = obsah.text.strip()
        
    return(atributy)

## Test

In [137]:
buh = scrape_film(slozka, "9013-kolja.html")
buh

{'Země původu': 'Česká republika,                                                             Velká Británie,                                                             Francie',
 'Copyright': '1996',
 'Rok výroby': '1995',
 'Minutáž': '108 min',
 'Režie': 'Jan Svěrák (476)',
 'Kategorie': 'film',
 'Žánr': 'komedie,                                                             smutná komedie',
 'Originální název': 'Kolja',
 'Český název': 'Kolja',
 'Anglický název': 'Kolya',
 'Pomocná režie': 'David Rauch (1810)',
 'Asistent režie': 'Slávek Horák (80123), Nikola Hejko (7510)',
 'Původní filmový námět': 'Pavel Taussig (11546)',
 'Scénář': 'Zdeněk Svěrák (1892)',
 'Dramaturg': 'Václav Šašek (1766), Anna Vášová (37301)',
 'Kamera': 'Vladimír Smutný (1772)',
 'Asistent kamery': 'Jan Carda (7004)',
 'Architekt': 'Miloš J. Kohout (5609)',
 'Návrhy kostýmů': 'Katarína Hollá (6534)',
 'Masky': 'František Příhoda (7109)',
 'Střih': 'Alois Fišárek (570)',
 'Zvuk': 'zvukový',
 'Producent': 'Jan Sv

## Todo

- víc odstavců anotace a obsahu
- skladatelé písní
- víc režisérů a žánrů do seznamu

## Konečně DataFrame

In [138]:
import pandas as pd

In [139]:
kolik = None

In [101]:
def frejm(slozka):
    df = pd.DataFrame()
    total = len(seznam_souboru(slozka)[:kolik])
    count = 0
    for s in seznam_souboru(slozka)[:kolik]:
        count = count + 1
        print(f"{count} / {total} / {s}")
        line = pd.DataFrame.from_dict([scrape_film(slozka, s)])
        df = pd.concat([df, line])
    df = df.reset_index(drop = True)
    return(df)

In [140]:
df = frejm("filmy")
df

1 / 4538 / 10144-tabor-padlych-zen.html
2 / 4538 / 10173-eastern-krvavy-hugo.html
3 / 4538 / 102282-o-jecminkovi.html
4 / 4538 / 102314-zimni-vila.html
5 / 4538 / 10242-narkoblues.html
6 / 4538 / 103341-jan-hus-mse-za-tri-mrtve-muze.html
7 / 4538 / 103354-obcansky-prukaz.html
8 / 4538 / 103358-7-dni-hrichu.html
9 / 4538 / 103360-vojta-lavicka-nahoru-a-dolu.html
10 / 4538 / 103369-kuky-se-vraci.html
11 / 4538 / 103372-habermannuv-mlyn.html
12 / 4538 / 103374-dobre-placena-prochazka.html
13 / 4538 / 103402-nebe-peklo.html
14 / 4538 / 103495-lidice.html
15 / 4538 / 103499-medvedi-ostrovy.html
16 / 4538 / 103509-dont-stop.html
17 / 4538 / 104213-az-na-veky.html
18 / 4538 / 10574-elfilm.html
19 / 4538 / 106663-dum.html
20 / 4538 / 106666-soukromy-vesmir.html
21 / 4538 / 106667-mamas-papas.html
22 / 4538 / 10781-karu-suda-medvedi-srdce.html
23 / 4538 / 10960-unos-domu.html
24 / 4538 / 1118-nachove-plachty.html
25 / 4538 / 11221-mechanika.html
26 / 4538 / 112328-jmenem-krale.html
27 / 4538 / 

205 / 4538 / 180001-vsechno-je-sracka.html
206 / 4538 / 180025-laska-v-hrobe.html
207 / 4538 / 180317-rekvalifikace.html
208 / 4538 / 180386-muzi-v-nadeji.html
209 / 4538 / 18042-ctyri-lasky.html
210 / 4538 / 18045-fimfarum-jana-wericha.html
211 / 4538 / 180492-bastardi-2.html
212 / 4538 / 180545-svata-ctverice.html
213 / 4538 / 18056-franta-nebojsa.html
214 / 4538 / 18058-splneny-sen.html
215 / 4538 / 180585-okresni-prebor-posledni-zapas-pepika-hnatka.html
216 / 4538 / 180618-probudim-se-vcera.html
217 / 4538 / 181233-ve-stinu-bohyne-matky-zeme.html
218 / 4538 / 181912-zneuzivany.html
219 / 4538 / 181928-skolni-vylet.html
220 / 4538 / 181956-co-chytnes-v-zite.html
221 / 4538 / 181960-osina.html
222 / 4538 / 181961-cesta.html
223 / 4538 / 181962-stoh.html
224 / 4538 / 18203-z-mesta-cesta.html
225 / 4538 / 182231-libas-jako-dabel.html
226 / 4538 / 18262-pupendo.html
227 / 4538 / 18264-jedna-ruka-netleska.html
228 / 4538 / 18265-utek-do-budina.html
229 / 4538 / 182826-laska-je-laska.html

413 / 4538 / 395110-smich-a-plac.html
414 / 4538 / 395111-vystavni-parkar-a-lepic-plakatu.html
415 / 4538 / 395112-to-nejlepsi-cislo.html
416 / 4538 / 395113-jarni-sen-stareho-mladence.html
417 / 4538 / 395114-hubicka.html
418 / 4538 / 395115-ponrepovo-kouzelnictvi.html
419 / 4538 / 395116-rudi-na-krtinach.html
420 / 4538 / 395117-rudi-na-zaletech.html
421 / 4538 / 395118-rudi-se-zeni.html
422 / 4538 / 395119-rudi-sportsman.html
423 / 4538 / 395120-sokove.html
424 / 4538 / 395121-ctyri-rocni-doby.html
425 / 4538 / 395122-dama-s-barzojem.html
426 / 4538 / 395123-falesny-hrac.html
427 / 4538 / 395124-pro-penize.html
428 / 4538 / 395125-zahadny-zlocin.html
429 / 4538 / 395126-zive-modely.html
430 / 4538 / 395127-americky-souboj.html
431 / 4538 / 395128-cholera-v-praze.html
432 / 4538 / 395129-estrella.html
433 / 4538 / 395130-faust.html
434 / 4538 / 395131-idyla-ze-stare-prahy.html
435 / 4538 / 395132-konec-milovani.html
436 / 4538 / 395133-pan-profesor-nepritel-zen.html
437 / 4538 / 3951

615 / 4538 / 395313-on-a-ona.html
616 / 4538 / 395314-zkouska.html
617 / 4538 / 395315-bud-pripraven.html
618 / 4538 / 395316-carovne-oci.html
619 / 4538 / 395317-hrabenka-filangieri.html
620 / 4538 / 395318-kdyz-srdce-promluvi.html
621 / 4538 / 395319-madame-golvery.html
622 / 4538 / 395320-muz-bez-srdce.html
623 / 4538 / 395321-palicova-dcera.html
624 / 4538 / 395322-pepanek-nezdara.html
625 / 4538 / 395323-pro-a-proti.html
626 / 4538 / 395324-problematicky-gentleman.html
627 / 4538 / 395325-rasputin.html
628 / 4538 / 395326-tu-ten-kamen.html
629 / 4538 / 395327-unos-bankere-fuxe.html
630 / 4538 / 395328-za-oponou-smrti.html
631 / 4538 / 395329-zahadny-pripad-galginuv.html
632 / 4538 / 395330-zavet-podivinova.html
633 / 4538 / 395331-bily-raj.html
634 / 4538 / 395332-chytte-ho.html
635 / 4538 / 395333-dama-z-baru.html
636 / 4538 / 395334-devce-z-hor.html
637 / 4538 / 395335-dvoji-zivot.html
638 / 4538 / 395336-hrichy-v-manzelstvi.html
639 / 4538 / 395337-jindra-hrabenka-ostrovinova.h

812 / 4538 / 395511-pili-a-pokrokem-k-uspechu.html
813 / 4538 / 395512-pradlenka-jeho-jasnosti.html
814 / 4538 / 395513-tajemstvi-lekarovo.html
815 / 4538 / 395514-utrpeni-sede-sestry.html
816 / 4538 / 395515-v-kropenate-lhote-hori.html
817 / 4538 / 395516-vendelinuv-ocistec-a-raj.html
818 / 4538 / 395517-vse-pro-lasku.html
819 / 4538 / 395518-za-rodnou-hroudu.html
820 / 4538 / 395519-rada-lekare.html
821 / 4538 / 395520-tonka-sibenice.html
822 / 4538 / 395521-film-se-sasou-rasilovem-a-karlem-faltysem.html
823 / 4538 / 395522-kdyz-struny-lkaji-nemecka-verze.html
824 / 4538 / 395523-on-a-jeho-sestra-nemecka-verze.html
825 / 4538 / 395524-afera-plukovnika-redla-nemecka-verze.html
826 / 4538 / 395525-c-a-k-polni-marsalek-francouzska-verze.html
827 / 4538 / 395526-zena-ktera-se-smeje.html
828 / 4538 / 395527-svet-bez-hranic.html
829 / 4538 / 395528-afera-plukovnika-redla.html
830 / 4538 / 395529-pana-bumlicka-novy-domov.html
831 / 4538 / 395530-pensionat-petersilia.html
832 / 4538 / 395531

...

3914 / 4538 / 401865-dejte-nam-kridla.html
3915 / 4538 / 401866-deset-sladkych-minut.html
3916 / 4538 / 401867-dilo-bedricha-hrozneho.html
3917 / 4538 / 401868-divotvorne-oko.html
3918 / 4538 / 401869-2-kaucuk.html
3919 / 4538 / 401869-2x-kaucuk.html
3920 / 4538 / 401870-emil-zatopek.html
3921 / 4538 / 401871-gregor-mendel.html
3922 / 4538 / 401872-hudebni-jaro.html
3923 / 4538 / 401873-chemie-v-kuchyni.html
3924 / 4538 / 401874-jak-se-dela-film.html
3925 / 4538 / 401875-junaci-v-prirode.html
3926 / 4538 / 401876-kapky-a-bubliny.html
3927 / 4538 / 401877-ladovi-furianti.html
3928 / 4538 / 401878-lide-a-parky.html
3929 / 4538 / 401879-miliony-na-dlazbach.html
3930 / 4538 / 401880-narodni-umelec-josef-lada.html
3931 / 4538 / 401881-nevime-dne.html
3932 / 4538 / 401882-obraz.html
3933 / 4538 / 401883-sklenena-suita.html
3934 / 4538 / 401884-to-jsou-bratri-v-triku.html
3935 / 4538 / 401885-cena-za-stesti.html
3936 / 4538 / 401886-letni-hokej.html
3937 / 4538 / 401887-podkarpatska-rus.html


4103 / 4538 / 402060-pribeh-tantry.html
4104 / 4538 / 402061-postizeni-muzikou.html
4105 / 4538 / 402062-casting-na-lasku.html
4106 / 4538 / 402063-sarkofag-pro-kralovnu.html
4107 / 4538 / 402064-meky.html
4108 / 4538 / 402065-prilis-osobni-znamost.html
4109 / 4538 / 402067-svina.html
4110 / 4538 / 402068-3bobule.html
4111 / 4538 / 402069-havel.html
4112 / 4538 / 402070-kralove-videa.html
4113 / 4538 / 402071-kiruna-prekrasny-novy-svet.html
4114 / 4538 / 402072-milenec-leta.html
4115 / 4538 / 402073-cesta-do-nemozna.html
4116 / 4538 / 402075-svet-podle-muchy.html
4117 / 4538 / 402077-poslouchej.html
4118 / 4538 / 402078-zenska-pomsta.html
4119 / 4538 / 402079-modelar.html
4120 / 4538 / 402080-mlsne-medvedi-pribehy-v-kinech.html
4121 / 4538 / 402081-v-siti-18.html
4122 / 4538 / 402082-alchymicka-pec.html
4123 / 4538 / 402083-jan-jedlicka-stopy-krajiny.html
4124 / 4538 / 402084-frem.html
4125 / 4538 / 402085-nebe.html
4126 / 4538 / 402086-nova-sichta.html
4127 / 4538 / 402087-zmatek.html

4301 / 4538 / 52996-praha-ocima.html
4302 / 4538 / 53009-karty-jsou-rozdany.html
4303 / 4538 / 53011-absolutni-laska.html
4304 / 4538 / 53015-obrazky-z-vyletu.html
4305 / 4538 / 53021-riziko.html
4306 / 4538 / 53045-sest-statecnych.html
4307 / 4538 / 53047-prvy-statocny-vlado-kral-ked-nie-tak-nie.html
4308 / 4538 / 53051-druhy-statocny-vojtech-masek-tomik.html
4309 / 4538 / 53055-treti-statocny-jaro-vojtek-rojkoviasnilci.html
4310 / 4538 / 53057-stvrty-statocny-tereza-kucerova-prazske-nokturno-aneb-komora.html
4311 / 4538 / 53059-piaty-statocny-martin-repka-zlodeji.html
4312 / 4538 / 53102-siesty-statocny-aurel-klimt-o-kouzelnem-zvonu.html
4313 / 4538 / 53125-vsichni-moji-blizci.html
4314 / 4538 / 53139-z-pekla-stesti.html
4315 / 4538 / 53143-venkovsky-ucitel.html
4316 / 4538 / 53335-klicek.html
4317 / 4538 / 53571-cernobila-v-barve.html
4318 / 4538 / 53944-kulicky.html
4319 / 4538 / 53961-hannibal-zrozeni.html
4320 / 4538 / 53971-tulaci.html
4321 / 4538 / 54020-zlati-uhori.html
4322 /

4501 / 4538 / 9099-kral-ubu.html
4502 / 4538 / 9101-co-delat-cesta-z-prahy-do-ceskeho-krumlova-aneb-jak-jsem-ses.html
4503 / 4538 / 9102-marian.html
4504 / 4538 / 9103-spiklenci-slasti.html
4505 / 4538 / 9104-kamenny-most.html
4506 / 4538 / 9124-byl-jednou-jeden-polda-ii-major-maisner-opet-zasahuje.html
4507 / 4538 / 91455-made-in-famu.html
4508 / 4538 / 9148-septej.html
4509 / 4538 / 9149-lotrando-a-zubejda.html
4510 / 4538 / 9151-maskin-zabil-koskina.html
4511 / 4538 / 91578-hodinu-nevis.html
4512 / 4538 / 91594-male-lzi.html
4513 / 4538 / 9168-rychle-pohyby-oci.html
4514 / 4538 / 91819-ocas-jesterky.html
4515 / 4538 / 91952-zaby.html
4516 / 4538 / 9214-ptak-ohnivak.html
4517 / 4538 / 9215-orbis-pictus.html
4518 / 4538 / 9222-modre-z-nebe.html
4519 / 4538 / 9238-suzanne.html
4520 / 4538 / 9241-hranice-stinu.html
4521 / 4538 / 9242-bumerang.html
4522 / 4538 / 9290-jmeno-kodu-rubin.html
4523 / 4538 / 9295-konto-separato.html
4524 / 4538 / 9296-lea.html
4525 / 4538 / 9358-passage.html
4

,Země původu,Copyright,Rok výroby,Premiéra,Minutáž,Režie,Kategorie,Žánr,Originální název,Český název,...,povídkový film,Povídky,Prodej práv,Podnázev anglický,Výtvarník pozadí,Výrobní skupina,Zákazník,Vývozní společnost,Rychlost,Přístupnost
0,"Slovensko, ...",1997,1997,premiéra 28. 9. 2000 /doporučená přístupnost o...,100 min,Laco Halama,film,"historický, ...",Tábor padlých žien,Tábor padlých žen,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Česká republika,1997,1997,nedistribuční premiéra 27. 4. 1997 ...,6 min,Aurel Klimt (1189),film,groteska,Eastern – Krvavý Hugo,Eastern – Krvavý Hugo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Česká republika,2003,2002,nedistribuční premiéra 18. 4. 2003 (televizní...,61 min,Milan Cieslar (3593),film,pohádka,O Ječmínkovi,O Ječmínkovi,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Česká republika,1999,1999,nedistribuční premiéra 24. 12. 1999 (televizn...,52 min,Kryštof Hanzlík (1055),film,pohádka,Zimní víla,Zimní víla,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Česká republika, ...",1997,1997,premiéra 10. 11. 2016 /přístupné bez omezení/,9 min,"Břetislav Pojar (3113), Ivan Vít (3030)",film,osvětový,Narkoblues,Narkoblues,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4533,Česká republika,1997,1997,premiéra 12. 8. 1997,NaN,Rodrigo Morales (3466),film,povídka,Hotel Sarajevo aneb Sny obyčejného svatebčana ...,Hotel Sarajevo aneb Sny obyčejného svatebčana ...,...,Zviditelnění,Hotel Sarajevo aneb Sny obyčejného svatebčana ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4534,Česká republika,1997,1997,premiéra 13. 11. 1997 /přístupný/ (kino 64 U H...,114 min,Hynek Bočan (2947),film,"dobrodružný, ...",Zdivočelá země,Zdivočelá země,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4535,Česká republika,1997,1997,nedistribuční premiéra 12. 6. 1997 ...,5 min,Galina Hůlová (7957),film,anekdota,Biograf,Biograf,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4536,Česká republika,1997,1997,premiéra 20. 11. 1997 /doporučená přístupnost ...,102 min,Petr Zelenka (4806),film,"komedie, ...",Knoflíkáři,Knoflíkáři,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df.columns

Index(['Země původu', 'Copyright', 'Rok výroby', 'Premiéra', 'Minutáž',
       'Režie', 'Kategorie', 'Žánr', 'Originální název', 'Český název',
       'Anglický název', 'Souběžný název', 'Pracovní název', 'Pomocná režie',
       'Asistent režie', 'Původní filmový námět', 'Scénář', 'Dramaturg',
       'Kamera', 'Asistent kamery', 'Architekt', 'Návrhy kostýmů', 'Masky',
       'Střih', 'Zvuk', 'Producent', 'Vedoucí výroby',
       'Zástupce vedoucího výroby', 'Odborný poradce', 'Typologie',
       'Distribuční slogan', 'Výrobce', 'S podporou', 'Nositelé copyrightu',
       'Distribuce', 'Délka', 'Distribuční nosič', 'Poměr stran', 'Barva',
       'Zvukový systém/formát', 'Verze', 'Mluveno', 'Podtitulky',
       'Úvodní/závěrečné titulky', 'Anotace', 'Obsah', 'Technický scénář',
       'Výtvarník', 'Animace', 'Triky', 'Postprodukce', 'Návrhy titulků',
       'Pedagogické vedení', 'Studentský film', 'Původní metráž',
       'Mezititulky', 'Vizuální efekty', 'Vedoucí produkce', 'Předloha',


## Export

In [141]:
df.to_json("filmy_raw.json", orient="records")